In [7]:
import pandas as pd
import numpy as np # Para -float('inf'), float('inf') se necessário
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

# Para widgets interativos no Jupyter
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Configurações do Pandas para melhor visualização (opcional)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


In [8]:
# Mapeamento fornecido das colunas
column_names = [
    'Project_ID', 'Project_Size', 'Team_Experience_Level', 'Estimated_Effort_Hours',
    'Actual_Effort_Hours', 'Risk_Score', 'Complexity_Level', 'Requirement_Stability',
    'Budget_Deviation_Percentage', 'Schedule_Deviation_Percentage', 'Defects_Introduced',
    'ML_Feasibility_Score'
]

# Carregar o dataset
try:
    # Assumindo que o arquivo CSV TEM uma linha de cabeçalho (ex: A, B, C...) que queremos substituir
    # Se o arquivo NÃO TEM cabeçalho, use header=None
    # Se os cabeçalhos já estão corretos no arquivo, use apenas pd.read_csv('seu_arquivo.csv')
    df = pd.read_csv('Reduced_Software_Project_Risk_Dataset_585.csv', header=0, names=column_names)
    # Se a linha acima não funcionar como esperado (por exemplo, a primeira linha de dados é perdida),
    # e seu arquivo não tem cabeçalhos, tente:
    # df = pd.read_csv('Reduced_Software_Project_Risk_Dataset_585.csv', header=None, names=column_names)

    print("Dataset carregado com sucesso.")
except FileNotFoundError:
    print("Erro: O arquivo 'Reduced_Software_Project_Risk_Dataset_585.csv' não foi encontrado.")
    print("Por favor, certifique-se de que o arquivo está no mesmo diretório que o notebook ou forneça o caminho completo.")
    df = pd.DataFrame() # Cria um DataFrame vazio para evitar erros subsequentes se o arquivo não for encontrado
except Exception as e:
    print(f"Ocorreu um erro ao ler o CSV: {e}")
    df = pd.DataFrame()

if not df.empty:
    print("\nPrimeiras 5 linhas do dataset:")
    display(df.head())

    print("\nInformações do dataset:")
    df.info()

    print("\nEstatísticas descritivas:")
    display(df.describe(include='all'))

    # Verificar se a coluna 'Risk_Score' existe
    if 'Risk_Score' not in df.columns:
        print("\nERRO CRÍTICO: A coluna 'Risk_Score' não foi encontrada após o carregamento.")
        print("Colunas disponíveis:", df.columns.tolist())
    else:
        print(f"\nColuna 'Risk_Score' encontrada. Tipo: {df['Risk_Score'].dtype}")

Dataset carregado com sucesso.

Primeiras 5 linhas do dataset:


,Project_ID,Project_Size,Team_Experience_Level,Estimated_Effort_Hours,Actual_Effort_Hours,Risk_Score,Complexity_Level,Requirement_Stability,Budget_Deviation_Percentage,Schedule_Deviation_Percentage,Defects_Introduced,ML_Feasibility_Score
0,522,Medium,High,1572,1887,9.34,High,Moderate,45.56,3.46,43,0.39
1,738,Large,Low,916,968,2.88,Medium,Stable,-4.93,8.07,35,0.20
2,741,Medium,High,675,1956,5.34,Low,Stable,-5.05,3.44,16,0.49
3,661,Medium,Medium,357,1953,4.70,Low,Unstable,-14.81,19.67,27,0.15
4,412,Large,Low,919,1045,2.77,Low,Moderate,-4.48,-1.49,29,0.32



Informações do dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 585 entries, 0 to 584
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Project_ID                     585 non-null    int64  
 1   Project_Size                   585 non-null    object 
 2   Team_Experience_Level          585 non-null    object 
 3   Estimated_Effort_Hours         585 non-null    int64  
 4   Actual_Effort_Hours            585 non-null    int64  
 5   Risk_Score                     585 non-null    float64
 6   Complexity_Level               585 non-null    object 
 7   Requirement_Stability          585 non-null    object 
 8   Budget_Deviation_Percentage    585 non-null    float64
 9   Schedule_Deviation_Percentage  585 non-null    float64
 10  Defects_Introduced             585 non-null    int64  
 11  ML_Feasibility_Score           585 non-null    float64
dtypes: float64(4), int64(4), 

,Project_ID,Project_Size,Team_Experience_Level,Estimated_Effort_Hours,Actual_Effort_Hours,Risk_Score,Complexity_Level,Requirement_Stability,Budget_Deviation_Percentage,Schedule_Deviation_Percentage,Defects_Introduced,ML_Feasibility_Score
count,585.000000,585,585,585.000000,585.000000,585.000000,585,585,585.000000,585.000000,585.000000,585.000000
unique,NaN,3,3,NaN,NaN,NaN,3,3,NaN,NaN,NaN,NaN
top,NaN,Medium,Medium,NaN,NaN,NaN,Medium,Moderate,NaN,NaN,NaN,NaN
freq,NaN,230,234,NaN,NaN,NaN,252,225,NaN,NaN,NaN,NaN
mean,484.400000,NaN,NaN,1078.572650,1110.656410,5.552684,NaN,NaN,13.934615,9.712496,23.482051,0.543368
std,287.588838,NaN,NaN,563.916204,547.001797,2.612347,NaN,NaN,20.146871,11.352017,14.071738,0.254716
min,1.000000,NaN,NaN,100.000000,102.000000,1.010000,NaN,NaN,-19.980000,-9.980000,0.000000,0.100000
25%,245.000000,NaN,NaN,585.000000,639.000000,3.180000,NaN,NaN,-3.680000,0.000000,11.000000,0.320000
50%,474.000000,NaN,NaN,1083.000000,1164.000000,5.600000,NaN,NaN,12.950000,9.830000,23.000000,0.550000
75%,717.000000,NaN,NaN,1574.000000,1615.000000,7.890000,NaN,NaN,31.980000,18.810000,36.000000,0.760000



Coluna 'Risk_Score' encontrada. Tipo: float64


In [9]:
if df.empty:
    print("DataFrame original está vazio. Pulando pré-processamento.")
else:
    df_processed = df.copy()

    # Remover Project_ID
    if 'Project_ID' in df_processed.columns:
        df_processed = df_processed.drop(columns=['Project_ID'])

    # --- Início da Lógica de Discretização Detalhada ---
    # 1. Converter colunas para numérico ANTES de discretizar (se necessário)
    cols_to_convert_to_numeric = ['Estimated_Effort_Hours', 'Actual_Effort_Hours', 'Risk_Score',
                                  'Budget_Deviation_Percentage', 'Schedule_Deviation_Percentage',
                                  'Defects_Introduced', 'ML_Feasibility_Score']
    for col in cols_to_convert_to_numeric:
        if col in df_processed.columns:
            if df_processed[col].dtype == 'object':
                df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
        else:
            print(f"Aviso: Coluna {col} esperada para conversão numérica não encontrada.")


    # 2. Discretizar Estimated_Effort_Hours
    if 'Estimated_Effort_Hours' in df_processed.columns and pd.api.types.is_numeric_dtype(df_processed['Estimated_Effort_Hours']):
        quantiles_eeh = df_processed['Estimated_Effort_Hours'].quantile([0.25, 0.5, 0.75]).values
        df_processed['Estimated_Effort_Hours_Cat'] = pd.cut(
            df_processed['Estimated_Effort_Hours'],
            bins=[-float('inf'), quantiles_eeh[0], quantiles_eeh[1], quantiles_eeh[2], float('inf')],
            labels=['Muito Baixo', 'Baixo', 'Médio', 'Alto'], include_lowest=True)
    else:
        print("Aviso: 'Estimated_Effort_Hours' não pôde ser discretizada.")

    # 3. Discretizar Actual_Effort_Hours
    if 'Actual_Effort_Hours' in df_processed.columns and pd.api.types.is_numeric_dtype(df_processed['Actual_Effort_Hours']):
        quantiles_aeh = df_processed['Actual_Effort_Hours'].quantile([0.25, 0.5, 0.75]).values
        df_processed['Actual_Effort_Hours_Cat'] = pd.cut(
            df_processed['Actual_Effort_Hours'],
            bins=[-float('inf'), quantiles_aeh[0], quantiles_aeh[1], quantiles_aeh[2], float('inf')],
            labels=['Muito Baixo', 'Baixo', 'Médio', 'Alto'], include_lowest=True)
    else:
        print("Aviso: 'Actual_Effort_Hours' não pôde ser discretizada.")

    # 4. Discretizar Risk_Score
    if 'Risk_Score' in df_processed.columns and pd.api.types.is_numeric_dtype(df_processed['Risk_Score']):
        df_processed['Risk_Score_Cat'] = pd.cut(
            df_processed['Risk_Score'], bins=[0, 3.3, 6.6, 10.00001],
            labels=['Baixo', 'Médio', 'Alto'], include_lowest=True)
    else:
        print("Aviso: 'Risk_Score' não pôde ser discretizada.")

    # 5. Discretizar Budget_Deviation_Percentage
    if 'Budget_Deviation_Percentage' in df_processed.columns and pd.api.types.is_numeric_dtype(df_processed['Budget_Deviation_Percentage']):
        bins_budget = [-float('inf'), -0.05, 0.05, float('inf')]
        labels_budget = ['Abaixo_Orcamento', 'Dentro_Orcamento', 'Acima_Orcamento']
        df_processed['Budget_Deviation_Cat'] = pd.cut(
            df_processed['Budget_Deviation_Percentage'], bins=bins_budget, labels=labels_budget, include_lowest=True)
    else:
        print("Aviso: 'Budget_Deviation_Percentage' não pôde ser discretizada.")

    # 6. Discretizar Schedule_Deviation_Percentage
    if 'Schedule_Deviation_Percentage' in df_processed.columns and pd.api.types.is_numeric_dtype(df_processed['Schedule_Deviation_Percentage']):
        bins_schedule = [-float('inf'), -0.05, 0.05, float('inf')]
        labels_schedule = ['Adiantado', 'No_Prazo', 'Atrasado']
        df_processed['Schedule_Deviation_Cat'] = pd.cut(
            df_processed['Schedule_Deviation_Percentage'], bins=bins_schedule, labels=labels_schedule, include_lowest=True)
    else:
        print("Aviso: 'Schedule_Deviation_Percentage' não pôde ser discretizada.")
        
    # 7. Discretizar Defects_Introduced
    if 'Defects_Introduced' in df_processed.columns and pd.api.types.is_numeric_dtype(df_processed['Defects_Introduced']):
        quantiles_defects = df_processed['Defects_Introduced'].quantile([0.25, 0.5, 0.75]).values
        bins_defects_list = [-1, quantiles_defects[0], quantiles_defects[1], quantiles_defects[2], float('inf')]
        unique_bins_defects = sorted(list(set(bins_defects_list)))
        num_categories_defects = len(unique_bins_defects) - 1
        labels_defects = []
        if num_categories_defects == 4: labels_defects = ['Muito Baixos', 'Baixos', 'Médios', 'Altos']
        elif num_categories_defects == 3: labels_defects = ['Baixos', 'Médios', 'Altos']
        elif num_categories_defects == 2: labels_defects = ['Baixos', 'Altos']
        elif num_categories_defects == 1: labels_defects = ['Unica_Categoria']
        
        if labels_defects:
            df_processed['Defects_Introduced_Cat'] = pd.cut(
                df_processed['Defects_Introduced'], bins=unique_bins_defects, labels=labels_defects,
                include_lowest=True, duplicates='drop')
        else:
            print("Aviso: 'Defects_Introduced' não pôde ser discretizada adequadamente. Poucos bins únicos.")
            df_processed['Defects_Introduced_Cat'] = 'Nao_Discretizado' # Categoria padrão
    else:
        print("Aviso: 'Defects_Introduced' não pôde ser discretizada.")

    # 8. Discretizar ML_Feasibility_Score
    if 'ML_Feasibility_Score' in df_processed.columns and pd.api.types.is_numeric_dtype(df_processed['ML_Feasibility_Score']):
        df_processed['ML_Feasibility_Score_Cat'] = pd.cut(
            df_processed['ML_Feasibility_Score'], bins=[-0.00001, 0.33, 0.66, 1.00001],
            labels=['Baixa', 'Média', 'Alta'], include_lowest=True)
    else:
        print("Aviso: 'ML_Feasibility_Score' não pôde ser discretizada.")

    # --- Fim da Lógica de Discretização Detalhada ---

    # Selecionar colunas para a Rede Bayesiana (TODAS as colunas na estrutura do modelo)
    # Esta lista deve ser derivada da `model_structure` que definiremos na próxima célula.
    # Por enquanto, vamos preparar uma lista com todas as colunas _Cat e as categóricas originais.
    
    categorical_cols_original = ['Project_Size', 'Team_Experience_Level', 'Complexity_Level', 'Requirement_Stability']
    newly_created_cat_cols = [col for col in df_processed.columns if col.endswith('_Cat')]
    
    all_potential_model_cols = categorical_cols_original + newly_created_cat_cols
    
    # Filtrar para garantir que todas as colunas existem em df_processed
    final_model_cols = [col for col in all_potential_model_cols if col in df_processed.columns]
    
    df_bayes = df_processed[final_model_cols].copy()

    # Tratar NaNs nas colunas finais (resultantes de 'coerce' ou falhas no cut)
    print("\nValores ausentes por coluna em df_bayes ANTES do tratamento final:")
    print(df_bayes.isnull().sum())

    for column in df_bayes.columns:
        if df_bayes[column].isnull().any():
            mode_val = df_bayes[column].mode()
            if not mode_val.empty:
                df_bayes[column] = df_bayes[column].fillna(mode_val[0])
                print(f"NaNs em '{column}' preenchidos com a moda: {mode_val[0]}")
            else: # Caso a coluna inteira seja NaN ou todos os valores sejam únicos e haja NaNs
                df_bayes[column] = df_bayes[column].fillna('Desconhecido')
                print(f"NaNs em '{column}' preenchidos com 'Desconhecido' (moda não encontrada).")


    print("\nPrimeiras 5 linhas do dataset processado (df_bayes):")
    display(df_bayes.head())

    print("\nInformações do dataset processado (df_bayes):")
    df_bayes.info()

    print("\nValores únicos por coluna em df_bayes (APÓS tratamento de NaNs):")
    for column in df_bayes.columns:
        print(f"Coluna '{column}': {df_bayes[column].unique()}")


Valores ausentes por coluna em df_bayes ANTES do tratamento final:
Project_Size                  0
Team_Experience_Level         0
Complexity_Level              0
Requirement_Stability         0
Estimated_Effort_Hours_Cat    0
Actual_Effort_Hours_Cat       0
Risk_Score_Cat                0
Budget_Deviation_Cat          0
Schedule_Deviation_Cat        0
Defects_Introduced_Cat        0
ML_Feasibility_Score_Cat      0
dtype: int64

Primeiras 5 linhas do dataset processado (df_bayes):


,Project_Size,Team_Experience_Level,Complexity_Level,Requirement_Stability,Estimated_Effort_Hours_Cat,Actual_Effort_Hours_Cat,Risk_Score_Cat,Budget_Deviation_Cat,Schedule_Deviation_Cat,Defects_Introduced_Cat,ML_Feasibility_Score_Cat
0,Medium,High,High,Moderate,Médio,Alto,Alto,Acima_Orcamento,Atrasado,Altos,Média
1,Large,Low,Medium,Stable,Baixo,Baixo,Baixo,Abaixo_Orcamento,Atrasado,Médios,Baixa
2,Medium,High,Low,Stable,Baixo,Alto,Médio,Abaixo_Orcamento,Atrasado,Baixos,Média
3,Medium,Medium,Low,Unstable,Muito Baixo,Alto,Médio,Abaixo_Orcamento,Atrasado,Médios,Baixa
4,Large,Low,Low,Moderate,Baixo,Baixo,Baixo,Abaixo_Orcamento,Adiantado,Médios,Baixa



Informações do dataset processado (df_bayes):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 585 entries, 0 to 584
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   Project_Size                585 non-null    object  
 1   Team_Experience_Level       585 non-null    object  
 2   Complexity_Level            585 non-null    object  
 3   Requirement_Stability       585 non-null    object  
 4   Estimated_Effort_Hours_Cat  585 non-null    category
 5   Actual_Effort_Hours_Cat     585 non-null    category
 6   Risk_Score_Cat              585 non-null    category
 7   Budget_Deviation_Cat        585 non-null    category
 8   Schedule_Deviation_Cat      585 non-null    category
 9   Defects_Introduced_Cat      585 non-null    category
 10  ML_Feasibility_Score_Cat    585 non-null    category
dtypes: category(7), object(4)
memory usage: 23.5+ KB

Valores únicos por coluna em df_bayes (

In [10]:
if 'df_bayes' not in locals() or df_bayes.empty:
    print("df_bayes não está definido ou está vazio. Não é possível treinar o modelo.")
    bayesian_model = None
    inference_engine = None
else:
    # Definir a estrutura da rede (relações Pai -> Filho)
    model_structure = [
        ('Project_Size', 'Estimated_Effort_Hours_Cat'),
        ('Complexity_Level', 'Estimated_Effort_Hours_Cat'),
        ('Team_Experience_Level', 'Estimated_Effort_Hours_Cat'),

        ('Complexity_Level', 'Defects_Introduced_Cat'),
        ('Team_Experience_Level', 'Defects_Introduced_Cat'),
        ('Requirement_Stability', 'Defects_Introduced_Cat'),

        ('Estimated_Effort_Hours_Cat', 'Actual_Effort_Hours_Cat'),
        ('Team_Experience_Level', 'Actual_Effort_Hours_Cat'),
        ('Complexity_Level', 'Actual_Effort_Hours_Cat'),

        ('Actual_Effort_Hours_Cat', 'Schedule_Deviation_Cat'),
        ('Requirement_Stability', 'Schedule_Deviation_Cat'),

        ('Actual_Effort_Hours_Cat', 'Budget_Deviation_Cat'),

        ('Schedule_Deviation_Cat', 'Risk_Score_Cat'),
        ('Budget_Deviation_Cat', 'Risk_Score_Cat'),
        ('Defects_Introduced_Cat', 'Risk_Score_Cat'),
        ('Complexity_Level', 'Risk_Score_Cat'),
        ('ML_Feasibility_Score_Cat', 'Risk_Score_Cat'),
        ('Requirement_Stability', 'Risk_Score_Cat'),
        ('Team_Experience_Level', 'Risk_Score_Cat')
    ]

    # Verificar se todas as colunas da estrutura existem em df_bayes
    all_nodes_in_structure = set()
    for parent, child in model_structure:
        all_nodes_in_structure.add(parent)
        all_nodes_in_structure.add(child)

    missing_cols = [node for node in all_nodes_in_structure if node not in df_bayes.columns]
    if missing_cols:
        print(f"ERRO CRÍTICO: As seguintes colunas definidas na estrutura não existem em df_bayes: {missing_cols}")
        print(f"Colunas disponíveis em df_bayes: {df_bayes.columns.tolist()}")
        print("Verifique o passo de pré-processamento e a criação das colunas _Cat.")
        bayesian_model = None # Impede a continuação
        inference_engine = None
    else:
        print("Todas as colunas da estrutura estão presentes em df_bayes.")
        # Usar apenas as colunas que estão na estrutura para treinar o modelo
        df_train = df_bayes[list(all_nodes_in_structure)].copy()

        bayesian_model = DiscreteBayesianNetwork(model_structure) # <-- MUDANÇA AQUI
        
        print("\nAjustando o modelo aos dados (aprendendo CPDs)...")
        # Certificar que não há NaNs no df_train antes do fit, pois o estimador pode não gostar.
        # O tratamento de NaNs em df_bayes já deveria ter cuidado disso.
        if df_train.isnull().values.any():
            print("AVISO: Existem valores NaN no df_train. Tentando preencher com a moda novamente para cada coluna...")
            for col_train in df_train.columns:
                if df_train[col_train].isnull().any():
                    mode_val_train = df_train[col_train].mode()
                    if not mode_val_train.empty:
                        df_train[col_train] = df_train[col_train].fillna(mode_val_train[0])
                    else:
                        df_train[col_train] = df_train[col_train].fillna('Desconhecido_fit') 
            if df_train.isnull().values.any():
                print("ERRO CRÍTICO: NaNs ainda presentes em df_train após tentativa de preenchimento. O 'fit' pode falhar.")


        bayesian_model.fit(data=df_train, estimator=MaximumLikelihoodEstimator)

        print("\nVerificando o modelo...")
        if bayesian_model.check_model():
            print("Modelo Bayesiano (DiscreteBayesianNetwork) construído e verificado com sucesso!")
            
            # Preparar o motor de inferência
            inference_engine = VariableElimination(bayesian_model)
            print("Motor de inferência (VariableElimination) pronto.")

            # Imprimir algumas CPDs para inspeção (opcional)
            print("\nCPD para Risk_Score_Cat:")
            display(bayesian_model.get_cpds('Risk_Score_Cat'))
            # print("\nCPD para Defects_Introduced_Cat:")
            # display(bayesian_model.get_cpds('Defects_Introduced_Cat'))

        else:
            print("Aviso: A verificação do modelo (DiscreteBayesianNetwork) encontrou problemas.")
            bayesian_model = None # Impede a continuação
            inference_engine = None

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'Budget_Deviation_Cat': 'O', 'Actual_Effort_Hours_Cat': 'O', 'Team_Experience_Level': 'C', 'Defects_Introduced_Cat': 'O', 'Requirement_Stability': 'C', 'Estimated_Effort_Hours_Cat': 'O', 'Risk_Score_Cat': 'O', 'Project_Size': 'C', 'ML_Feasibility_Score_Cat': 'O', 'Schedule_Deviation_Cat': 'O', 'Complexity_Level': 'C'}


Todas as colunas da estrutura estão presentes em df_bayes.

Ajustando o modelo aos dados (aprendendo CPDs)...

Verificando o modelo...
Modelo Bayesiano (DiscreteBayesianNetwork) construído e verificado com sucesso!
Motor de inferência (VariableElimination) pronto.

CPD para Risk_Score_Cat:


<TabularCPD representing P(Risk_Score_Cat:3 | Budget_Deviation_Cat:3, Complexity_Level:3, Defects_Introduced_Cat:4, ML_Feasibility_Score_Cat:3, Requirement_Stability:3, Schedule_Deviation_Cat:3, Team_Experience_Level:3) at 0x23ff0bc2780>

In [11]:
if 'inference_engine' in locals() and inference_engine is not None:
    print("\n--- Exemplo de Inferência Básica ---")
    try:
        # Verifique os valores exatos das categorias em df_bayes para usar nas evidências.
        # Ex: df_bayes['Project_Size'].unique()
        
        evidence_example = {
            'Project_Size': 'Large',  # Ajuste se necessário
            'Team_Experience_Level': 'Low', # Ajuste se necessário
            'Complexity_Level': 'High'  # Ajuste se necessário
        }
        
        # Verificar se os valores da evidência são válidos
        valid_evidence = True
        for key, value in evidence_example.items():
            if key in df_bayes.columns: # Verifica se a coluna existe
                if value not in df_bayes[key].unique():
                    print(f"AVISO: Valor de evidência '{value}' para '{key}' não é um estado válido. Estados válidos: {df_bayes[key].unique()}")
                    # Poderia invalidar a evidência ou tentar um valor padrão. Por agora, apenas avisa.
            else:
                print(f"AVISO: Coluna de evidência '{key}' não encontrada no modelo/dados.")
                valid_evidence = False # Não podemos usar esta evidência

        if valid_evidence:
            query_result_basic = inference_engine.query(
                variables=['Risk_Score_Cat'],
                evidence=evidence_example
            )
            print(f"\nProbabilidade de Risk_Score_Cat dadas as evidências: {evidence_example}")
            print(query_result_basic)
        else:
            print("Inferência básica não pode ser realizada devido a evidências inválidas.")

    except Exception as e:
        print(f"Ocorreu um erro durante a inferência básica: {e}")
        print("Verifique se as variáveis de evidência e seus estados existem no modelo.")
else:
    print("Motor de inferência não está pronto. Pule o exemplo de inferência.")


--- Exemplo de Inferência Básica ---

Probabilidade de Risk_Score_Cat dadas as evidências: {'Project_Size': 'Large', 'Team_Experience_Level': 'Low', 'Complexity_Level': 'High'}
+-----------------------+-----------------------+
| Risk_Score_Cat        |   phi(Risk_Score_Cat) |
+=======================+=======================+
| Risk_Score_Cat(Alto)  |                0.3332 |
+-----------------------+-----------------------+
| Risk_Score_Cat(Baixo) |                0.3470 |
+-----------------------+-----------------------+
| Risk_Score_Cat(Médio) |                0.3197 |
+-----------------------+-----------------------+


In [12]:
def perform_sensitivity_analysis(target_variable, evidence_variable, base_evidence, model_inference_engine, data_frame_for_states):
    """
    Realiza uma análise de sensibilidade simples.
    """
    print(f"\n--- Análise de Sensibilidade para '{target_variable}' em relação a '{evidence_variable}' ---")

    if evidence_variable not in data_frame_for_states.columns:
        print(f"Erro: Variável de evidência '{evidence_variable}' não encontrada no dataframe de estados.")
        return {}
    
    if not hasattr(model_inference_engine, 'query'):
        print("Erro: Motor de inferência inválido.")
        return {}

    possible_states = data_frame_for_states[evidence_variable].unique()
    print(f"Valores possíveis para '{evidence_variable}': {possible_states}")

    current_base_evidence = base_evidence.copy()
    if evidence_variable in current_base_evidence:
        del current_base_evidence[evidence_variable]

    results_sensitivity = {}
    output_html = f"<h3>Análise de Sensibilidade: {target_variable} vs {evidence_variable}</h3>"
    output_html += f"<p>Evidências base fixas: {current_base_evidence}</p>"
    output_html += "<table><tr><th>Estado de " + evidence_variable + "</th><th>Distribuição de " + target_variable + "</th></tr>"


    for state in possible_states:
        current_evidence = current_base_evidence.copy()
        current_evidence[evidence_variable] = state
        
        try:
            query_result = model_inference_engine.query(
                variables=[target_variable],
                evidence=current_evidence
            )
            results_sensitivity[state] = query_result
            
            # Formatar para HTML
            prob_text = "<ul style='list-style-type:none; padding-left:0;'>"
            target_var_name_sens = query_result.variables[0]
            for i, state_name_sens in enumerate(query_result.state_names[target_var_name_sens]):
                prob_text += f"<li>{state_name_sens}: {query_result.values[i]*100:.2f}%</li>"
            prob_text += "</ul>"
            output_html += f"<tr><td>'{state}'</td><td>{prob_text}</td></tr>"

        except Exception as e:
            error_msg = f"Erro ao consultar com '{evidence_variable}' = '{state}': {e}"
            print(error_msg)
            results_sensitivity[state] = error_msg
            output_html += f"<tr><td>'{state}'</td><td>{error_msg}</td></tr>"
    
    output_html += "</table>"
    display(HTML(output_html))
    print("\n--- Fim da Análise de Sensibilidade ---")
    return results_sensitivity

if 'inference_engine' in locals() and inference_engine is not None and 'df_bayes' in locals():
    print("\nExecutando exemplos de Análise de Sensibilidade...")
    
    # Usar a moda das colunas de df_bayes como cenário base
    base_scenario_evidence_sens = {}
    # Selecionar apenas colunas que são "entradas" comuns para o cenário base
    # e que não serão a 'evidence_variable' na análise atual.
    input_cols_for_scenario = ['Project_Size', 'Team_Experience_Level', 'Complexity_Level', 
                               'Requirement_Stability', 'ML_Feasibility_Score_Cat']
    
    for col_scenario in input_cols_for_scenario:
        if col_scenario in df_bayes.columns:
             base_scenario_evidence_sens[col_scenario] = df_bayes[col_scenario].mode()[0]
    
    print(f"\nCenário base para outras evidências na análise de sensibilidade: {base_scenario_evidence_sens}")

    # 1. Sensibilidade do Risk_Score_Cat em relação ao Complexity_Level
    perform_sensitivity_analysis(
        target_variable='Risk_Score_Cat',
        evidence_variable='Complexity_Level', # Esta será variada
        base_evidence=base_scenario_evidence_sens.copy(), # Passa uma cópia
        model_inference_engine=inference_engine,
        data_frame_for_states=df_bayes
    )

    # 2. Sensibilidade do Risk_Score_Cat em relação ao Team_Experience_Level
    perform_sensitivity_analysis(
        target_variable='Risk_Score_Cat',
        evidence_variable='Team_Experience_Level', # Esta será variada
        base_evidence=base_scenario_evidence_sens.copy(), # Passa uma cópia
        model_inference_engine=inference_engine,
        data_frame_for_states=df_bayes
    )
else:
    print("Motor de inferência ou df_bayes não está pronto. Pule a análise de sensibilidade.")


Executando exemplos de Análise de Sensibilidade...

Cenário base para outras evidências na análise de sensibilidade: {'Project_Size': 'Medium', 'Team_Experience_Level': 'Medium', 'Complexity_Level': 'Medium', 'Requirement_Stability': 'Moderate', 'ML_Feasibility_Score_Cat': 'Média'}

--- Análise de Sensibilidade para 'Risk_Score_Cat' em relação a 'Complexity_Level' ---
Valores possíveis para 'Complexity_Level': ['High' 'Medium' 'Low']


Estado de Complexity_Level,Distribuição de Risk_Score_Cat
'High',Alto: 26.65%Baixo: 29.76%Médio: 43.59%
'Medium',Alto: 41.40%Baixo: 26.09%Médio: 32.51%
'Low',Alto: 50.25%Baixo: 14.03%Médio: 35.73%



--- Fim da Análise de Sensibilidade ---

--- Análise de Sensibilidade para 'Risk_Score_Cat' em relação a 'Team_Experience_Level' ---
Valores possíveis para 'Team_Experience_Level': ['High' 'Low' 'Medium']


Estado de Team_Experience_Level,Distribuição de Risk_Score_Cat
'High',Alto: 48.72%Baixo: 29.27%Médio: 22.01%
'Low',Alto: 58.65%Baixo: 13.81%Médio: 27.54%
'Medium',Alto: 41.40%Baixo: 26.09%Médio: 32.51%



--- Fim da Análise de Sensibilidade ---


In [13]:
if 'inference_engine' in locals() and inference_engine is not None and 'df_bayes' in locals():
    print("\n--- Configurando Interface Interativa para Previsão de Risco ---")

    # Definir os campos de entrada para o usuário
    # Estas devem ser colunas presentes em df_bayes e que fazem sentido como entrada do usuário
    interactive_input_fields = {
        'Project_Size': None,
        'Team_Experience_Level': None,
        'Complexity_Level': None,
        'Requirement_Stability': None,
        'ML_Feasibility_Score_Cat': None
    }

    valid_interactive_fields = {}
    for field, _ in interactive_input_fields.items():
        if field in df_bayes.columns:
            unique_states = sorted(list(df_bayes[field].dropna().unique()))
            if unique_states:
                 valid_interactive_fields[field] = widgets.Dropdown(options=unique_states, description=f"{field.replace('_', ' ').title()}:")
            else:
                print(f"Aviso: Sem estados únicos para '{field}' após dropna(). Widget não será criado.")
        else:
            print(f"Aviso: Campo '{field}' para widget interativo não encontrado em df_bayes.")

    # Botão de Previsão
    predict_button = widgets.Button(description="Prever Risco", button_style='info', icon='search')
    
    # Área de Saída para os resultados
    output_area = widgets.Output()

    def on_predict_button_clicked(b):
        with output_area:
            clear_output(wait=True) # Limpa a saída anterior
            
            evidence_interactive = {}
            print("Coletando dados dos widgets...")
            valid_input = True
            for field_name, widget_item in valid_interactive_fields.items():
                if widget_item.value is None: # Checa se algum dropdown não tem valor (pouco provável com defaults)
                    print(f"Erro: O campo '{field_name}' não tem um valor selecionado.")
                    display(HTML(f"<p style='color:red;'>Por favor, selecione um valor para {field_name}.</p>"))
                    valid_input = False
                    break
                evidence_interactive[field_name] = widget_item.value
            
            if not valid_input:
                return

            if not evidence_interactive:
                display(HTML("<p style='color:red;'>Nenhuma evidência fornecida. Não é possível prever.</p>"))
                return

            print(f"Evidências para predição: {evidence_interactive}")
            
            try:
                query_result_interactive = inference_engine.query(
                    variables=['Risk_Score_Cat'],
                    evidence=evidence_interactive
                )
                
                # Preparar HTML para o resultado
                result_html = "<h3>Resultado da Previsão de Risco:</h3>"
                result_html += "<ul>"
                target_var_name_interactive = query_result_interactive.variables[0]
                for i, state_name_interactive in enumerate(query_result_interactive.state_names[target_var_name_interactive]):
                    prob = query_result_interactive.values[i] * 100
                    result_html += f"<li><b>{state_name_interactive}</b>: {prob:.2f}%</li>"
                result_html += "</ul>"
                display(HTML(result_html))

            except Exception as e:
                error_message = f"Ocorreu um erro durante a predição interativa: {e}"
                print(error_message)
                display(HTML(f"<p style='color:red;'>{error_message}</p>"))

    predict_button.on_click(on_predict_button_clicked)

    # Exibir os widgets
    if valid_interactive_fields:
        # Criar uma VBox para organizar os widgets
        widget_list = list(valid_interactive_fields.values())
        input_widgets_box = widgets.VBox(widget_list)
        
        display(HTML("<h2>Previsão de Risco Interativa</h2>"))
        display(HTML("<p>Selecione os valores para as características do projeto e clique em 'Prever Risco'.</p>"))
        display(input_widgets_box)
        display(predict_button)
        display(output_area)
    else:
        print("Nenhum widget de entrada válido foi criado. Interface interativa não pode ser exibida.")

else:
    print("Motor de inferência ou df_bayes não está pronto. Pule a interface interativa.")


--- Configurando Interface Interativa para Previsão de Risco ---


Button(button_style='info', description='Prever Risco', icon='search', style=ButtonStyle())

Output()